<h1>전이 학습(Transfer Learning)을 이용해서 이미지 분류 모델 생성하기</h1>

1. 왼쪽에서 폴더란을 열고 [마우스 우클릭]-[새 폴더]를 클릭해 폴더를 생성하고, 폴더 안에 [마우스 우클릭]-[업로드]를 클릭해 이미지들을 업로드 합니다.

2. 상단 메뉴 [런타임]-[모두 실행]을 클릭해 이 프로그램을 실행시킵니다. 학습을 시켜야 하므로 5분 이상은 소요될 겁니다 ㅠㅠ

3. 맨 밑의 출력 결과에 'Running on http://1a2b3c4d5e.ngrok.io' 라고 나오는 부분이 있습니다. 위와 같이 나오는 링크를 TLECommander.py 파일에서 맨 윗줄에 선언되어 있는 link 변수에 대입합니다. copy 변수에 True를 대입하면 복사, False를 대입하면 이동합니다.

4. TLECommander.py 파일과 같이 있는 images 폴더에 분류시킬 이미지들을 넣어놓고 TLECommander.py 프로그램을 실행시키면 이미지들을 분류해줍니다. 분류 결과는 result 폴더에서 확인할 수 있습니다.

** TLECommander.py 파일<br>
라이브러리: os, shutil<br>
버전: Python 3.6.5<br>
운영체제: MacOS Big Sur 11.3<br><br>


<출처><br>
3분 만에 만드는 인공지능 서비스 개발 강좌: 마동석/김종국/이병헌 분류기, 나동빈,<br>
https://www.youtube.com/watch?v=Lu93Ah2h9XA&t=6s &<br>
https://github.com/ndb796/CNN-based-Celebrity-Classification-AI-Service-Using-Transfer-Learning

In [ ]:
import os

querys = []
imgext = ['.jpeg', '.jpg', '.png', '.gif', '.bmp', '.tif']

for folder in os.listdir('./'):
    if os.path.isdir(folder):
        for file in os.listdir('./' + folder):
            if os.path.isfile('./' + folder + '/' + file):
                found = False
                for i in range(0, len(imgext), 1):
                    if imgext[i] in file.lower():
                        querys.append(folder)
                        found = True
                        break
                
                if found:
                    break

print('분류 기준: ', querys)

분류 기준:  ['kotori', 'honoka']


In [ ]:
!apt install fonts-nanum -y

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
plt.rc('font', family='NanumBarunGothic')
matplotlib.font_manager._rebuild()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (10.1 MB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [ ]:
import os
import shutil

directory_list = [
    './custom_dataset/train/',
    './custom_dataset/test/',
]

for directory in directory_list:
    if not os.path.isdir(directory):
        os.makedirs(directory)

def dataset_split(query, train_cnt):
    for directory in directory_list:
        if not os.path.isdir(directory + '/' + query):
            os.makedirs(directory + '/' + query)
    cnt = 0
    for file_name in os.listdir(query):
        for i in range(0, len(imgext), 1):
            if imgext[i] in file_name.lower():
                if cnt < train_cnt:
                    print(f'[Train Dataset] {file_name}')
                    shutil.copy(query + '/' + file_name, './custom_dataset/train/' + query + '/' + file_name)
                    # shutil.rmtree('')
                else:
                    print(f'[Test Dataset] {file_name}')
                    shutil.copy(query + '/' + file_name, './custom_dataset/test/' + query + '/' + file_name)

            cnt += 1

In [ ]:
for i in range(0, len(querys), 1):
  dataset_split(querys[i], 30)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time

# cuda:0 or cpu
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

In [ ]:
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir = './custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=4, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=4, shuffle=True, num_workers=4)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
def imshow(input, title):
    input = input.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    input = std * input + mean
    input = np.clip(input, 0, 1)
    plt.imshow(input)
    plt.title(title)
    plt.show()


iterator = iter(train_dataloader)

inputs, classes = next(iterator)
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
model = models.resnet34(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 3)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [ ]:
num_epochs = 50
model.train()
start_time = time.time()

for epoch in range(num_epochs):
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.

    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#0 Loss: 1.3960 Acc: 50.0000% Time: 5.1018s
#1 Loss: 0.6517 Acc: 70.0000% Time: 10.0222s
#2 Loss: 0.4214 Acc: 80.0000% Time: 14.9244s
#3 Loss: 0.4021 Acc: 70.0000% Time: 19.8216s
#4 Loss: 0.0562 Acc: 100.0000% Time: 24.7395s
#5 Loss: 0.3263 Acc: 80.0000% Time: 29.6939s
#6 Loss: 0.0157 Acc: 100.0000% Time: 34.6536s
#7 Loss: 0.2913 Acc: 80.0000% Time: 39.6346s
#8 Loss: 0.0069 Acc: 100.0000% Time: 44.5593s
#9 Loss: 0.1613 Acc: 90.0000% Time: 49.5957s
#10 Loss: 0.4282 Acc: 80.0000% Time: 54.5889s
#11 Loss: 0.2078 Acc: 90.0000% Time: 59.5464s
#12 Loss: 0.0087 Acc: 100.0000% Time: 64.4391s
#13 Loss: 0.6516 Acc: 80.0000% Time: 69.3707s
#14 Loss: 1.2408 Acc: 70.0000% Time: 74.3473s
#15 Loss: 0.0064 Acc: 100.0000% Time: 79.2445s
#16 Loss: 0.4728 Acc: 80.0000% Time: 84.1723s
#17 Loss: 0.0069 Acc: 100.0000% Time: 89.0527s
#18 Loss: 0.0521 Acc: 100.0000% Time: 93.8761s
#19 Loss: 0.3142 Acc: 90.0000% Time: 98.7412s
#20 Loss: 0.1628 Acc: 90.0000% Time: 103.5386s
#21 Loss: 0.0345 Acc: 100.0000% Time:

In [ ]:
model.eval()
start_time = time.time()

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
        imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

    epoch_loss = running_loss / len(test_datasets)
    epoch_acc = running_corrects / len(test_datasets) * 100.
    print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
from PIL import Image
import os

imgName = ''

for file in os.listdir(querys[0]):
    for i in range(0, len(imgext), 1):
        if imgext[i] in file.lower():
            imgName = file
            break

image = Image.open(querys[0] + '/' + imgName)
image = transforms_test(image).unsqueeze(0).to(device)

with torch.no_grad():
    outputs = model(image)
    _, preds = torch.max(outputs, 1)
    imshow(image.cpu().data[0], title='예측 결과: ' + class_names[preds[0]]

In [ ]:
!pip install flask-ngrok

In [ ]:
import io
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request


def get_prediction(image_bytes):
    image = Image.open(io.BytesIO(image_bytes))
    image = transforms_test(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
        imshow(image.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

    return class_names[preds[0]]


app = Flask(__name__)


@app.route('/', methods=['POST'])
def predict():
    if request.method == 'POST':
        file = request.files['file']
        image_bytes = file.read()

        class_name = get_prediction(image_bytes=image_bytes)
        print("결과:", {'class_name': class_name})
        return jsonify({'class_name': class_name})

* 클라이언트 테스트 방법

<pre>
curl -X POST -F file=@{이미지 파일명} {Ngrok 서버 주소}
</pre>

* 사용 예시

<pre>
curl -X POST -F file=@dongseok.jpg http://c4cdb8de3a35.ngrok.io/
</pre>

In [ ]:
run_with_ngrok(app)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1ee53eb92e02.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/May/2021 00:40:02] "POST / HTTP/1.1" 200 -


결과: {'class_name': 'kotori'}


...

결과: {'class_name': 'kotori'}


127.0.0.1 - - [12/May/2021 00:40:12] "POST / HTTP/1.1" 200 -


결과: {'class_name': 'kotori'}


127.0.0.1 - - [12/May/2021 00:40:16] "POST / HTTP/1.1" 200 -


결과: {'class_name': 'kotori'}
